# 概要

畳込みと線形時不変システムのインパルス応答について，サンプルコードの実行を通して理論を確認してみる．<br/>
連続時間信号の畳込みは無限範囲の積分を含み，コンピュータ上での実装はほぼ不可能なので，離散時間信号を対象とする．<br/>
準備として，次のコマンドを実行して関連ファイルをダウンロードしておいてください．<br/>
また，このページでは上から順にサンプルコードを実行してください（途中から実行するとエラーが生じます．）

In [ ]:
!git clone https://github.com/knakamura1982/signal_processing.git

# 離散時間信号の畳込み

離散時間信号 $x[n]$ と $y[n]$ の畳込みを $z[n]$ とおくと，$z[n]$ は
$$
z[n]=x[n]*y[n]=\sum_{k=-\infty}^\infty x[n-k]y[k]
$$

で定義される．<br/>
コンピュータでは無限和は扱えないし，そもそも無限の長さを持つ信号を扱うこともできないので，<br/>
以降では $x[n],\ y[n]$ の長さをそれぞれ $N_x,\ N_y$ とし，
$$
\left\{
\begin{align}
x[n] &= 0 \quad (0<n \; または \; N_x \le n) \\
y[n] &= 0 \quad (0<n \; または \; N_y \le n)
\end{align}
\right.
$$
と考える．<br/>
このとき，$y[k]=0$ の範囲は $z[n]$ の計算から除外できるので，
$$
\left\{
\begin{align}
a &= \max(0,\ n-N_x+1) \\
b &= \min(n,\ N_y-1)
\end{align}
\right.
$$
とおき，$a \le b$ の場合に限り
$$
z[n]=\sum_{k=a}^b x[n-k]y[k]
$$
として計算すれば良い．<br/>
なお，$a<b$ が満たされるのは $0 \le n < N_x + N_y - 1$ のときであるから，<br/>
$z[n]$ は長さ $N_z = N_x + N_y - 1$ の信号として実装すれば良い．

以上のことを次のサンプルコードで試してみる．


In [ ]:
import numpy as np
from signal_processing.utils import show_discrete_signal


### 設定項目：ここから ###

# 畳込み計算対象の離散時間信号
x = [1, 3, 2, -2, -1, -1, 1]  # x: 長さ 7 の離散時間信号，配列で表現 
y = [1, 2, 1, -1]  # y: 長さ 4 の離散時間信号，配列で表現

### 設定項目：ここまで ###


# 有限長の離散時間信号の畳込みを実現する関数
# 離散時間信号 x, y は配列で与えられており，各々の配列のサイズを信号長 Nx, Ny とみなす．
# また，計算結果は配列 z として返す．
def convolution(x, y):

  Nx = len(x)  # 離散時間信号 x の長さ
  Ny = len(y)  # 離散時間信号 y の長さ
  Nz = Nx + Ny - 1  # 計算結果（畳込み後の信号）の長さ

  # 長さ Nz の空配列を用意
  z = np.zeros(Nz)

  # 畳込み計算
  for n in range(0, Nz):
    a = max(0, n - Nx + 1)  # k の下限
    b = min(n, Ny - 1)  # k の上限
    z[n] = 0
    for k in range(a, b+1):  # k=a～b で足し合わせ
      z[n] += x[n-k] * y[k]

  # 計算結果を返す
  return z


# 畳込みを計算
z = convolution(x, y)
Nz = len(z)

# 配列 x, y, z をそれぞれ離散時間信号と解釈し，n = 0～Nz の範囲をグラフ表示
show_discrete_signal(x, nrange=[0, Nz], vrange=[-10, 10], title='x[n]')
show_discrete_signal(y, nrange=[0, Nz], vrange=[-10, 10], title='y[n]')
show_discrete_signal(z, nrange=[0, Nz], vrange=[-10, 10], title='z[n]')

# 線形時不変システムのインパルス応答

線形時不変な離散時間信号処理システムでは，入力とインパルス応答の畳込みにより出力が表現できる．<br/>
このことを確かめてみる．

## システムの例

例として，
$$
y[n] = L[x[n]] = \frac{1}{2}x[n] - \frac{1}{3}x[n-1] + \frac{1}{6}x[n-2]
$$
で表されるシステム $L$ を考える．

## インパルス応答

インパルス応答とは，単位インパルス信号
$$
\delta[n] = \left\{
  \begin{array}{ll}
  1 & (n=0) \\
  0 & (n \neq 0)
  \end{array}
\right.
$$
を $L$ に入力したときの出力である．<br/>
以下のコードで実際に求めてみる．

In [ ]:
import numpy as np
from signal_processing.utils import show_discrete_signal


# 離散時間信号処理システム
# 式： y[n] = 1/2 x[n] - 1/3 x[n-1] + 1/6 x[n-2]
# 配列（入力の離散時間信号）を受け取って配列（出力の離散時間信号）を返す関数として定義
def L(x):

  # 2ステップ前までの入力を参照して各時刻の出力値を決定する都合上，出力が入力よりも 2 だけ長くなることを踏まえ，
  # 最初から配列 x の最後に 0 を二つ追加して信号長を 2 だけ長くしておく
  x = np.asarray([(x[n] if 0 <= n and n < len(x) else 0) for n in range(len(x)+2)])  # 最後の「+2」で二つ 0 を追加している

  N = len(x)  # 信号長（この時点で本来の入力信号長 + 2 になっている）

  # 長さ N の空配列を用意
  y = np.zeros(N)

  # 各時刻 n における出力信号値を計算
  for n in range(0, N):
    y[n] = (1/2) * x[n] - (1/3) * x[n-1] + (1/6) * x[n-2]

  # 計算結果を返す
  return y


# とりあえず，長さ 5 くらいの配列として離散時間の単位インパルス信号を定義
delta = [1, 0, 0, 0, 0]

# delta を L に入力し，インパルス応答 h を取得
h = L(delta)

# 配列 delta, h をそれぞれ離散時間信号と解釈し，n = 0～10 の範囲をグラフ表示
show_discrete_signal(delta, nrange=[0, 10], title='delta[n]')
show_discrete_signal(h, nrange=[0, 10], title='h[n]')

## 入力信号からの直接計算

適当な入力信号をシステム $L$ に入力し，出力信号を直接計算してみる．

In [ ]:
import numpy as np
from signal_processing.utils import show_discrete_signal


### 設定項目：ここから ###

# 入力信号として用いる離散時間信号を適当な配列として用意
x = [1, 3, 2, -2, -1, -1, 1]  # 畳込み計算時に使用したものと同じ信号

### 設定項目：ここまで ###


# x を L に入力し，出力信号 y を取得
y = L(x)

# 配列 delta, h をそれぞれ離散時間信号と解釈し，n = 0～10 の範囲をグラフ表示
show_discrete_signal(x, nrange=[0, 10], vrange=[-3, 3], title='x[n]')
show_discrete_signal(y, nrange=[0, 10], vrange=[-3, 3], title='y[n]')

## インパルス応答との畳込み

入力 $x[n]$ とインパルス応答 $h[n]$ の畳込みを計算し，上記の $y[n]$ と比較してみる．<br/>
両者が一致することが確認できる．

In [ ]:
import numpy as np
from signal_processing.utils import show_discrete_signal


# 入力信号 x とインパルス応答 h の畳込みを計算する（結果を y2 としておく）．
y2 = convolution(x, h)

# 配列 y, y2 をそれぞれ離散時間信号と解釈し，n = 0～10 の範囲をグラフ表示
show_discrete_signal(y, nrange=[0, 10], vrange=[-3, 3], title='y[n]')
show_discrete_signal(y2, nrange=[0, 10], vrange=[-3, 3], title='x[n] * h[n]')